# UPF Placement as SFCR

In [1]:
import pickle
import time
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
from pyomo.core import Var

## Defining Aux. Functions

In [2]:
def import_data(dir_data, file_name):
    """THis function imports data that it is in dir_data+file_name"""
    with open (dir_data+file_name, "rb") as fp:
        return pickle.load(fp)

In [3]:
def export_data(file_name, file_data):
    """This function allows saving the information contained in file_data in a file named file_data"""
    with open(file_name, "wb") as fp:
        pickle.dump(file_data, fp)
    return

## Importing Data

In [6]:
dir_data = 'path_to_dataset'  

# Number of PDU sessions or SFCR)
S = import_data(dir_data, "S") 

# Session information
N_sfc_type = import_data(dir_data, "N_sfc_type_S"+str(S))           # Type of SFC
Dict_Tsfc_s = import_data(dir_data, "Dict_Tsfc_s_S"+str(S))         # Matrix with VNFs in SFC: 1 if VNF f ∈ Fs in SFCR s ∈ S is of type t ∈ T
Dict_C_s = import_data(dir_data, "Dict_C_s_S"+str(S))               # Computing resources required by SFCR s ∈ S
Dict_Lreq_s = import_data(dir_data, "Dict_Lreq_s_S"+str(S))         # E2E latency requirement of SFCR s ∈ S
Dict_beta_s = import_data(dir_data, "Dict_beta_s_S"+str(S))         # Bandwidth capacity required by SFCR s ∈ S
Source_BS = import_data(dir_data, "Source_BS_S"+str(S))             # Access node (rs ∈ Nr) of SFCR s ∈ S


# Nodes
# Candidate locations
Nc = import_data(dir_data, "ListMECInstalled")           # candidate coordenadas
Nc = sorted(Nc, key=lambda k: [k[1], k[0]])             
Dict_C_c = import_data(dir_data, "Dict_C_c")             # Processing capacity at candidate location c ∈ Nc

# access nodes
Nr = import_data(dir_data, "BS_pos")
Nr = sorted(Nr, key=lambda k: [k[1], k[0]])

# Set of all nodes
N = Nc + Nr  


#Importing paths data 
Edges = import_data(dir_data,"Edges")                               # dict with key nodes_id (en & ran) and value latency
Dict_Links_Capacity = import_data(dir_data,"Dict_Links_Capacity")   # dict with key nodes_id (en & ran) and value bandwidth
Paths = import_data(dir_data,"Paths")
Paths_nm = import_data(dir_data,"Paths_nm")                         # P_n,m: dict with key nodes_id (en & ran) and value list of paths between nodes n and m
Paths_links_mapping = import_data(dir_data,'Paths_links_mapping')   # H^p_uv: 1 if path p ∈ P is mapped to physical link (u,v) ∈ E


# VNFs Data
Dict_C_t = import_data(dir_data,'Dict_C_t')             # Processing capacity of VNF of type t ∈ T         
Dict_D_proc = import_data(dir_data,'Dict_D_proc')       # d_t: Processing delay of VNF of type t ∈ T
M_t_imported = import_data(dir_data,'M_t')              # M_t: Maximum number of instances of type t ∈ T. Compute upon service demands and VNF capacity
T = import_data(dir_data,'T')                           # T: Types of VNFs
T_extended = T+1                                        # Extending T dataset to include RANs as an special VNF type
T_dn = import_data(dir_data,'T_dn')                     # Processing time of data network nodes


# Preparing the Data

In [7]:
# Splitting the set of links between candidate nodes and ran-candidates

Edge_candidates = {}
Edge_rans_candidates = {}

for k,v in Edges.items():
    if k[0] <= len(Nc) and k[1] <= len(Nc):
        Edge_candidates[k] = v
    else:
        Edge_rans_candidates[k] = v

if len(Edge_candidates) + len(Edge_rans_candidates) != len(Edges):
    print('Error: There was something wrong in the splitting process')
    raise

True

## Defining the general parameters for each type SFC

####  Types of PDU Sessions/ SFCR

+ Type 1:     
------| PSA |------
      
      
+ Type 2:     
------|I-UPF|-------| PSA |------
      
      
+ Type 3:     
------| M-UPF |-----|PSA |-----
               \̣____|PSA |-----
               
NOTE: All this types of VNFs has as initial VNF the RAN. In other words, the source nodes are considered as a type of VNFs. In the SFCR model the RANs are the last VNF and have t =4

### SFCR Parameters:
+ N_sfc_type: Number of CSFR types (i.e., 3)
+ B_sfc_type: Number of branches for each SFCR type
+ F_sfc_type: Number of VNFs for each SFCR type
+ K_sfc_type: Binary Parameter indicating if a VNF is common to all the branches (1) or not (0)
+ T_sfc_type: Parameter indicating the type of VNF f in for each SFCR type
+ P_fbs_sfc_type: Parameter indicating if VNF f is present in branch b for each SFCR type
+ O_sbfg_sfc_type: Parameter indicating if VNF f goes before g in branch b for each SFCR type

### SFCR Requirements:
+ L_s: Latency requirement of service s
+ C_s: Capacity demand of servise s
+ Source_s: Source access node/AP of service s (List_ran_user (s,f,t,m,c) )
+ Bw_s: Bandwidth demand of service s

In [10]:
#  Parameters for each type of SFCR

if N_sfc_type==3:                              # T: Number of VNFs types 1:PSA,  2:IUPF,3: M-IUPF, 4:RAN
    Ran_type = 4
    B_sfc_type = [1,1,2]                       # No. of branches in each type of SFC
    F_sfc_type = [[1,2],[1,2,3],[1, 2, 3, 4]]  # VNFs in each type of SFC
    T_sfc_type = [[1,4],[2,1,4],[3,1,1,4]]     # Type for each VNF in SFC 
    K_sfc_type = [[1,1],[1,1,1],[1,0,0,1]]     # Indicates if a VNF instance is common to all the branches or not
    P_fbs_sfc_type = [[[1,1]], 
                      [[1,1,1]], 
                      [[1,1,0,1],[1,0,1,1]]]   # Indicates if a VNF is present or not in a branch: Index[type,branch,vnf]
    O_sbfg_sfc_type  = [[[[0,0], [1,0]]],
                        [[[0,1,0],[0,0,0],[1,0,0]]],
                        [[[0,1,0,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]],
                        [[0,0,1,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]]]]  #Indicates if a VNF f goes before g in a branch

if N_sfc_type==2:                               # T: Number of VNFs types 1:PSA,  2:IUPF, 3:RAN
    Ran_type = 3
    B_sfc_type = [1,1]                          # No. of branches in each type of SFC
    F_sfc_type = [[1,2],[1,2,3]]                # VNFs in each type of SFC
    T_sfc_type = [[1,3],[2,1,3]]                # Type for each VNF in SFC 
    K_sfc_type = [[1,1],[1,1,1]]                # Indicates if a VNF instance is common to all the branches or not
    P_fbs_sfc_type = [[[1,1]], 
                      [[1,1,1]]]                # Indicates if a VNF is present or not in a branch: Index[type,branch,vnf]
    O_sbfg_sfc_type  = [[[[0,0], [1,0]]],
                        [[[0,1,0],[0,0,0],[1,0,0]]]]  #Indicates if a VNF f goes before g in a branch

#### Defining PDU sessions  parameters

In [11]:
#Generating the data of each session PDU (demand, A-UPF number, BS source)
List_F_s = []
List_T_s = []
List_K_s = []
List_P_s = []
List_O_s = []
List_B_s = []
List_R_s = []  # It has for each session the tuple (s,f,t,m,c)

for k,v in Dict_Tsfc_s.items():
    List_F_s.append(F_sfc_type[v-1])
    List_T_s.append(T_sfc_type[v-1])
    List_K_s.append(K_sfc_type[v-1])
    List_P_s.append(P_fbs_sfc_type[v-1])
    List_O_s.append(O_sbfg_sfc_type[v-1])
    List_B_s.append(B_sfc_type[v-1])
    List_R_s.append((k, len(F_sfc_type[v-1]), Ran_type, Nr.index(Source_BS[k])+1, Nr.index(Source_BS[k])+1+len(Nc)))

# Pyomo model 

In [13]:
# Creating the model 
model = ConcreteModel()

## Defining the Sets

In [14]:
#set of PDU sessions
model.del_component( 'S')
model.S = RangeSet(1, S)

#Set of candidates nodes
model.del_component( 'Nc')     
model.Nc = RangeSet(1, len(Nc))

#Set of ran nodes
model.del_component( 'Nr')     
model.Nr = RangeSet(len(Nc)+1, len(Nc)+len(Nr))

#Set of all the nodes: aggregation points+candidates nodes
model.del_component( 'N')     
model.N = RangeSet(1, len(N))

# Set of network edges. It includes the links between Nc+APs and Nc
model.del_component( 'E')
model.del_component( 'E_index')
model.E = Set(dimen=2, initialize=list(Edges.keys()), ordered= True)
# model.E = Set(dimen=2, initialize=list(Links_Latency.keys()), ordered= True)

# Set of network edges. It includes the links between Nc and Nc
model.del_component( 'E_NcxNc')
model.del_component( 'E_NcxNc_index')    
model.E_NcxNc = Set(dimen=2, initialize=list(Edge_candidates.keys()), ordered= True)
# model.E_NcxNc = Set(dimen=2, initialize=list(Links_Latency_NcxNc.keys()), ordered= True)

# Set of network edges. It includes the links between APs and Nc
model.del_component( 'E_NrxNc')
model.del_component( 'E_NrxNc_index')
model.E_NrxNc = Set(dimen=2, initialize=list(Edge_rans_candidates.keys()), ordered= True)
# model.E_NrxNc = Set(dimen=2, initialize=list(Links_Latency_NrxNc.keys()), ordered= True)

# Set with the combination of all the paths (n,m,h)
model.del_component( 'Paths')
model.del_component( 'Paths_index')
model.del_component( 'Paths_index_index_0')
model.Paths = Set(dimen=3, initialize=Paths.keys())

# Set with the combination of all endpoints (n,m)
Dict_Path_nm = {}
for k,v in Paths_nm.items():
    Dict_Path_nm[k]= [i+1 for i in range(v)]
    
model.del_component( 'Paths_nm')
model.del_component( 'Paths_nm_index')    
model.Paths_nm = Set(dimen=2, initialize=Dict_Path_nm.keys())

# Set with the number of paths bertween all endpoints (n,m)
model.del_component( 'Paths_nm_h')
model.del_component( 'Paths_nm_h_index')    
model.Paths_nm_h = Set(model.Paths_nm, initialize=Dict_Path_nm)

#Set of VNFs types (UPFs)
model.del_component( 'T')
model.T = RangeSet(1, T)

#Set of extended VNFs types (UPFs+RAN)
model.del_component( 'T_extended')
model.T_extended = RangeSet(1, T_extended)

# Set of available number of VNF instances for each type of VNF
Dict_M_t = {}
for k,v in M_t.items():
    Dict_M_t[k] = [i+1 for i in range(v)]
    
model.del_component( 'M_t')
model.M_t = Set(model.T_extended, initialize= Dict_M_t) 

# set TM 
model.del_component( 'TM')
model.del_component( 'TM_index')
List_TM = [(k,v1) for k,v in  Dict_M_t.items() for v1 in v if k!=4]
model.TM = Set(dimen=2, initialize= List_TM)

model.del_component( 'TM_extended')
model.del_component( 'TM_extended_index')
List_TM_extended = [(k,v1) for k,v in  Dict_M_t.items() for v1 in v]
model.TM_extended = Set(dimen=2, initialize= List_TM_extended)

#Sets of VNFs (basic and extended) forming a SFCR
Dict_VNF_s = {}
Dict_VNF_s_extended = {}
for s in model.S:
    Dict_VNF_s[s] = List_F_s[s-1][:-1]
    Dict_VNF_s_extended[s] = List_F_s[s-1]

model.del_component( 'F')
model.F = Set(model.S, initialize= Dict_VNF_s)

model.del_component( 'F_extended')
# model.F = Set(model.S, initialize= Dict_VNF_s_extended)
model.F_extended = Set(model.S, initialize= Dict_VNF_s_extended)

# SEt of SFCR and VNFs combined (s,f)
model.del_component( 'SF')
model.del_component( 'SF_index')
SF = [(s, i) for s in model.S for i in List_F_s[s-1][:-1]]  # List of tuples (user_id, vnf_id)
model.SF = Set(initialize= SF, ordered=True)

# SEt of SFCR and extended VNFs combined (s,f)
model.del_component( 'SF_extended')
model.del_component( 'SF_extended_index')
SF_extended =[(s, i) for s in model.S for i in List_F_s[s-1]]  # List of tuples (user_id, vnf_id)
# model.SF = Set(initialize= SF_extended, ordered=True)
model.SF_extended = Set(initialize= SF_extended, ordered=True)

# FGS is a list with [s,f,g] combination between any two VNFs f and g of each SFCR s
model.del_component( 'SFG')
model.del_component( 'SFG_index')
model.del_component( 'SFG_index_index_0')
SFG = [(s,f,g) for s in model.S for f in model.F[s] for g in model.F[s]]
model.SFG = Set(dimen=3, initialize= SFG)

# FGS_extended is a list with [s,f,g] combination between any two extended VNFs f and g of each SFCR s
model.del_component( 'SFG_extended')
model.del_component( 'SFG_extended_index')
model.del_component( 'SFG_extended_index_index_0')
SFG_extended = [(s,f,g) for s in model.S for f in model.F_extended[s] for g in model.F_extended[s]]
# model.SFG = Set(dimen=3, initialize= SFG)
model.SFG_extended = Set(dimen=3, initialize= SFG_extended)

# Set of branches in a SFC s
model.del_component( 'B_s')
model.del_component( 'B_s_index')
Dict_Branches={}
for s in model.S:
    Dict_Branches[s] = [b+1 for b in range(List_B_s[s-1])]
model.B_s = Set(model.S, initialize= Dict_Branches)

#Set with the combinations between sfcr and branches (s,b)
SB = [(s,b) for s in model.S for b in model.B_s[s] ]
model.del_component( 'SB')
model.del_component( 'SB_index')
model.SB = Set(dimen=2, initialize=SB)

# Set with the combinations between SFC s, Branches b and VNFs f (s,b,f)
model.del_component( 'SBF')
model.del_component( 'SBF_index')
model.del_component( 'SBF_index_index_0')
SBF = [(s,b,f) for s in model.S for b in model.B_s[s] for f in model.F[s]]
model.SBF = Set (dimen=3, initialize=SBF)

# Set with the combinations between SFC s, Branches b and extended VNFs f (s,b,f) for extended VNF set (RAN as a VNF)
model.del_component( 'SBF_extended')
model.del_component( 'SBF_extended_index')
model.del_component( 'SBF_extended_index_index_0')
SBF_extended = [(s,b,f) for s in model.S for b in model.B_s[s] for f in model.F_extended[s]]
# model.SBF = Set (dimen=3, initialize=SBF)
model.SBF_extended = Set (dimen=3, initialize=SBF_extended)

## Defining the Parameters

### Network Parameters

In [15]:
# VNF t capacity    
model.del_component( 'C_t')
model.C_t = Param(model.T_extended, initialize=Dict_C_t)

#MaxNumber of VNF instances for each type (M_t)
model.del_component( 'MaxM_t')
model.MaxM_t = Param(model.T_extended, initialize=M_t)

# Server capacity
model.del_component( 'C_c') 
model.C_c = Param(model.Nc, initialize=Dict_C_c)

# Node Supported Type of VNF
Dict_S_nt = {}  #dict with keys (n,t) and binary values, been 1 if node n support VNF type t, 0 otherwise
for n in model.N:
    for t in model.T_extended:
        if n <= len(Nc) and t!=Ran_type:
            Dict_S_nt[(n,t)]=1
        elif n>len(Nc) and t==Ran_type:  #ran nodes only support RANs(t=4)
            Dict_S_nt[(n,t)]=1
        else:
            Dict_S_nt[(n,t)]=0
            
model.del_component( 'NodeSupportType')
model.del_component( 'NodeSupportType_index')
model.NodeSupportType = Param(model.N, model.T_extended, initialize=Dict_S_nt)

# Propagation Latency in a path. Includes all the links between APs and Nc (N)
model.del_component( 'D_prop')
model.del_component( 'D_prop_index')
model.D_prop = Param(model.Paths, initialize=Paths)

#Processing Latency for each VNF type
model.del_component( 'D_proc')
model.D_proc = Param(model.T_extended, initialize=Dict_D_proc)

# Link Bandwidth
model.del_component( 'Link_bw')
model.del_component( 'Link_bw_index')
model.Link_bw = Param(model.E, initialize=Dict_Links_Capacity)

# Path_link mapping
model.del_component( 'W_nmh_uv')
model.del_component( 'W_nmh_uv_index')
model.del_component( 'W_nmh_uv_index_index_0')
model.del_component( 'W_nmh_uv_index_index_0_index_0')
model.del_component( 'W_nmh_uv_index_index_0_index_0_index_0')

Paths_links_mapping_dict = {}

for k,v in Paths_links_mapping.items():
    for k1,v1 in v.items():
        Paths_links_mapping_dict[(k[0],k[1], k[2], k1[0], k1[1])]=v1

model.W_nmh_uv = Param(model.Paths, model.E, initialize=Paths_links_mapping_dict)

### PDU Session Parameters

In [16]:
# SFCR processing demand 
model.del_component( 'C_s')
model.C_s = Param(model.S, initialize=Dict_C_s)

# SFCR latency demand 
model.del_component( 'Lreq_s')
model.Lreq_s = Param(model.S, initialize=Dict_Lreq_s)

#Link_Bandwidth demand
model.del_component( 'beta_s')
model.beta_s = Param(model.S, initialize=Dict_beta_s)

#Number of VNFs per sessions
Num_VNF_s = {}
for s in model.S:
    Num_VNF_s[s] = len(List_F_s[s-1])

model.del_component( 'Num_VNF_s')   
model.Num_VNF_s = Param(model.S, initialize=Num_VNF_s)  # for the moment it includes the ran

#Number of VNFs of each type in a SFCR
Dict_Num_VNFTypes_s = {}
for s in model.S:
    for t in model.T_extended:
        count= 0
        for element in List_T_s[s-1]:
            if t==element: count+=1
        Dict_Num_VNFTypes_s[(s,t)] = count

model.del_component( 'Num_VNFTypes_s')        
model.del_component( 'Num_VNFTypes_s_index')        
model.Num_VNFTypes_s = Param(model.S, model.T_extended, initialize=Dict_Num_VNFTypes_s)

#K_[s,f] = 1 if VNF f of SFC is s is common for all the branches in sfc s
Dict_K_s={}
for s,f in model.SF:
    Dict_K_s[(s,f)]= List_K_s[s-1][f-1]

model.del_component( 'K_sf')    
model.del_component( 'K_sf_index')    
model.K_sf = Param(model.SF, initialize=Dict_K_s)

Dict_K_s_ext={}
for s,f in model.SF_extended:
    Dict_K_s_ext[(s,f)]= List_K_s[s-1][f-1]

model.del_component( 'K_sf_extended')    
model.del_component( 'K_sf_extended_index')    
model.K_sf_extended = Param(model.SF_extended, initialize=Dict_K_s_ext)

# VNF presence in a branch P[s,b,f]= 1 if vnf f is present in branch b of sfc s
P_s= {}
for s,b,f in model.SBF_extended: 
    P_s[(s,b,f)] = List_P_s[s-1][b-1][f-1]

model.del_component( 'P_sbf')
model.del_component( 'P_sbf_index')
model.del_component( 'P_sbf_index_index_0')    
model.P_sbf = Param(model.SBF_extended, initialize=P_s)

#T_stf= 1 if VNF f in SFCs is of type t
Dict_T_sft ={} #key=(s,f,t), value= 1 if f is of type t, 0 otherwise
for s,f in model.SF_extended:
# for s,f in model.SF:
    for t in model.T_extended:
        if t == List_T_s[s-1][f-1]:
            Dict_T_sft[(s,f,t)]= 1
        else:
            Dict_T_sft[(s,f,t)]= 0
            
model.del_component( 'T_sft')
model.del_component( 'T_sft_index')
model.del_component( 'T_sft_index_index_0')    
model.T_sft = Param(model.SF_extended, model.T_extended, initialize=Dict_T_sft)

#O_sbfg= 1 if vnf f goes before g in branch b of sfc s
Dict_O_sbfg = {}
for s,b in model.SB:
    for f in model.F[s]:        
        for g in model.F[s]:
            Dict_O_sbfg[(s,b,f,g)]= List_O_s[s-1][b-1][f-1][g-1]
            
model.del_component( 'SBFG')
model.del_component( 'SBFG_index')
model.del_component( 'SBFG_index_index_0')    
model.del_component( 'SBFG_index_index_0_index_0')
model.SBFG = Set(dimen=4, initialize=Dict_O_sbfg.keys())

model.del_component( 'O_sbfg')
model.del_component( 'O_sbfg_index')
model.del_component( 'O_sbfg_index_index_0')    
model.del_component( 'O_sbfg_index_index_0_index_0')
model.O_sbfg = Param(model.SBFG,  initialize=Dict_O_sbfg)

Dict_O_sbfg_extended = {}
for s,b in model.SB:
    for f in model.F_extended [s]:
        for g in model.F_extended[s]:
            Dict_O_sbfg_extended[(s,b,f,g)]= List_O_s[s-1][b-1][f-1][g-1]
            
model.del_component( 'SBFG_extended')
model.del_component( 'SBFG_extended_index')
model.del_component( 'SBFG_extended_index_index_0')    
model.del_component( 'SBFG_extended_index_index_0_index_0')
model.SBFG_extended = Set(dimen=4, initialize=Dict_O_sbfg_extended.keys())

model.del_component( 'O_sbfg_extended')
model.del_component( 'O_sbfg_extended_index')
model.del_component( 'O_sbfg_extended_index_index_0')    
model.del_component( 'O_sbfg_extended_index_index_0_index_0')
model.O_sbfg_extended = Param(model.SBFG_extended,  initialize=Dict_O_sbfg_extended)

## Defining the Variables

In [17]:
# x[m,t,c]=1 if instance m of VNF type t is deployed at node c
model.del_component( 'x')
model.del_component( 'x_index')
model.del_component( 'x_index_index_0')
model.x = Var(model.TM_extended, model.N, within=Binary, initialize=0)

# z[s,f,sm,t,c]=1 if VNF f of SFCR s is assigned to instance m of VNF type t is deployed at node c
model.del_component( 'z')
model.del_component( 'z_index')
model.del_component( 'z_index_index_0')
model.del_component( 'z_index_index_0_index_0')
model.del_component( 'z_index_index_0_index_0_index_0')
model.z = Var(model.SF_extended, model.TM_extended, model.N, within=Binary, initialize=0)

# a[s,b,f,m,t,c] = 1 if the VNF f requested by the branch b of the SFC s is assigned to m,t,c VNF instance
model.del_component( 'a')
model.del_component( 'a_index')
model.del_component( 'a_index_index_0')
model.del_component( 'a_index_index_0_index_0')
model.del_component( 'a_index_index_0_index_0_index_0')
model.del_component( 'a_index_index_0_index_0_index_0_index_0')
model.a = Var(model.SBF_extended, model.TM_extended, model.N, within=Binary, initialize=0)

# y[s,b,f,g,n,m,h]= 1 if path h between nodes (i,j) is used to route traffic between VNFs f and g in branch b 
                    # of SFCR s 
model.del_component( 'y')
model.del_component( 'y_index')
model.del_component( 'y_index_index_0')
model.del_component( 'y_index_index_0_index_0')
model.del_component( 'y_index_index_0_index_0_index_0')
model.del_component( 'y_index_index_0_index_0_index_0_index_0')
model.del_component( 'y_index_index_0_index_0_index_0_index_0_index_0')
model.y = Var(model.SBFG_extended, model.Paths, within=Binary, initialize=0)

# delta[s,b,f,g,i,j]=1 if virtual link (i,j) may route traffic between VNFs f and g of SFCR s. It's a variable used for linearizing
model.del_component( 'delta')
model.del_component( 'delta_index')
model.del_component( 'delta_index_index_0')
model.del_component( 'delta_index_index_0_index_0')
model.del_component( 'delta_index_index_0_index_0_index_0')
model.del_component( 'delta_index_index_0_index_0_index_0_index_0')
model.delta = Var( model.SBFG_extended,model.Paths_nm,  within=Binary, initialize=0)
# model.delta = Var( model.SBFG, model.E, within=Binary, initialize=0)

# w[c]= 1 if node c is open
model.del_component( 'w') 
model.w = Var(model.Nc,  within=Binary, initialize=0)

## Defining the Objective

Objective functions:

Active nodes: $\sum \limits_{n \in N_c} w_{n}$

VNF number:   $\sum\limits_{i \in I_t}\sum \limits_{\substack{t\in T,\\ t\neq 0}}  \sum \limits_{n \in N_c} x_{i,t,n}$

Routing:      $\sum\limits_{f,g\in F_s^+}\sum\limits_{ b \in B_s}\sum\limits_{ s \in S}\sum\limits_{ p \in P}  d_{p} \cdot y_{p}^{f,g,b,s}$

In [18]:
model.del_component( 'Objective')

def Objective_rule(model):
    Num_upfs = sum(model.x[t,m,c] for t,m in model.TM for c in model.Nc)
    Activation = sum( model.w[c] for c in model.Nc)
    latency_oneway = sum(model.D_prop[n,m,h]*model.y[s,b,f,g,n,m,h]  for s,b in model.SB  
                         for f in model.F_extended[s] for g in model.F_extended[s] for n,m,h in model.Paths)
    
    # Normalizing
    Var1 = sum(M_t.values()) - M_t[Ran_type]
    Var2 = round(sum(Dict_C_c.values())/min(Dict_C_t.values()))
#     Max_Num_Instances = min(Var1, Var2)
    Num_upfs = Num_upfs/Max_Num_Instances
    Max_Num_Instances = Var2
    Branches = sum(List_B_s)
    latency_oneway = latency_oneway/(100*Branches)
    Activation = Activation/ len(Nc)
    
    w1,w2,w3 = 0.4,0.4,0.2          # weight components
    obj= w1*Num_upfs + w2*Activation + w3*latency_oneway
    
    return obj

model.Objective = Objective(rule=Objective_rule, sense=minimize)

In [19]:
# model.Objective.display()

## Defining the Constraints

##### C1: Node Capacity

$\sum \limits_{t\in T}\sum \limits_{m\in M_t}C_t*x_{m,t,c}<= C_c  \hspace{10 pt} \forall c \in N_c$

In [20]:
model.del_component( 'NodeCapacity')

def NodeCapacity_rule(model,c):
    return sum(model.C_t[t]*(model.x[t,m,c]) for t,m in model.TM) <= model.C_c[c]

model.NodeCapacity = Constraint(model.Nc, rule=NodeCapacity_rule)

In [21]:
# model.NodeCapacity.display()

##### C2: VNF Capacity

$ \sum \limits_{s\in S}\sum \limits_{f \in F_s} C_s[ K_{s}^f \cdot z_{t,i,n}^{s,f}  \leq C_t  \hspace{20 pt} \forall i \in I_t, \forall t \in T, t \neq 0, \forall n \in N_c $

In [22]:
model.del_component( 'VNFCapacity')
model.del_component( 'VNFCapacity_index')
model.del_component( 'VNFCapacity_index_index_0')

def VNFCapacity_rule(model,t,m,c):
    return sum(round(model.C_s[s],2)*model.z[s,f,t,m,c] for s,f in model.SF) <= model.C_t[t]


model.VNFCapacity = Constraint(model.TM, model.Nc, rule=VNFCapacity_rule)

In [23]:
# model.VNFCapacity.display()

##### C3: Link Capacity

$ \sum\limits_{s\in S} \sum\limits_{b\in B_s} \sum\limits_{f,g\in F_s^+}\sum\limits_{p\in P} \beta_s \cdot y_{p}^{s,b,f,g} \cdot W_{u,v}^{p}\leq \beta_{u,v} \hspace{20 pt} \forall (u,v) \in E $

In [24]:
model.del_component( 'LinkCapacity')
model.del_component( 'LinkCapacity_index')

def LinkCapacity_rule(model, i,j):
   
    link_cap = (sum(model.beta_s[s]*model.y[s,b,f,g,n,m,h]* model.W_nmh_uv[n,m,h,i,j] 
                    for s,b,f,g in model.SBFG_extended for n,m,h in model.Paths))
                    
    return link_cap <= model.Link_bw[i,j]

model.LinkCapacity = Constraint(model.E, rule=LinkCapacity_rule)


In [25]:
# model.LinkCapacity.display()

##### C4: Maximum number of intances of a type t
In the article, the maximum number of instances is defined as I_t

$ \sum \limits_{i \in I_t}\sum \limits_{n \in N_c} x_{i,t,n} \leq I_t \hspace{20 pt} \forall t \in  T; t \neq 0$

In [26]:
model.del_component( 'MaxNUmVNFType')

def MaxNUmVNFType_rule(model,t):
    return sum(model.x[t,m,c] for m in model.M_t[t] for c in model.Nc) <= model.MaxM_t[t]

model.MaxNUmVNFType = Constraint(model.T, rule=MaxNUmVNFType_rule)

In [27]:
# model.MaxNUmVNFType.display()

##### C5: Mapping of a VNF Instance 

Each VNf instance of a type must be mapped to only one server

$ \sum \limits_{n \in N_c} x_{i,t,n} \leq 1  \hspace{20 pt}   \forall i  \in  I_t, \forall t \in  T; t \neq 0$

In [28]:
model.del_component( 'MappingVNFInstance')
model.del_component( 'MappingVNFInstance_index')

def MappingVNFInstance_rule(model,t, m):
    return sum(model.x[t,m,c] for c in model.Nc) <= 1

model.MappingVNFInstance = Constraint(model.TM, rule=MappingVNFInstance_rule)

#### C6: Node support vnf type and must be open

An instance of a VNF type can be deployed on a node only if this type of VNF is supported by the node

$ x_{i,t,n} \leq w_n \cdot V_n^t \hspace{20 pt}  \forall i  \in  I_t, \forall t  \in  T, \forall n  \in  N$

In [30]:
model.del_component( 'NodeSupportTypeDeploy')
model.del_component( 'NodeSupportTypeDeploy_index')
model.del_component( 'NodeSupportTypeDeploy_index_index_0')

def NodeSupportTypeDeploy_rule(model,t,m,c):
    if t==Ran_type and c > len(Nc):             # If the type of vnf is a RAN as well as the node
        if m==(c-len(Nc)):
            return model.x[t,m,c]==1            # To match RAN instance with its number
        else:
            return model.x[t,m,c]==0
    else:    
        return model.x[t,m,c] <= model.w[n]*model.NodeSupportType[c,t]    

model.NodeSupportTypeDeploy = Constraint(model.TM_extended, model.N, rule=NodeSupportTypeDeploy_rule)

#### C: Node open

Forces a candidate to be closed if it has not mapped any VNF instance

$ \sum \limits_{\forall t \in T} \sum \limits_{\forall i \in I_t}  x_{t,i,c} \geq w_{c}   \hspace{30pt}   \forall c \in N$

In [ ]:
model.del_component( 'Opening')

def Opening_rule(model,c):
    return sum(model.x[t,i,c] for t,i in model.TM)>=model.w[c]

model.Opening = Constraint(model.Nc,  rule=Opening_rule)

##### C7: Mapping of VNF service equested by a SFCR

Each VNf service requested by a SFCR must be mapped to exactly one VNF instance 

$ \sum\limits_{ i \in I_t} \sum\limits_{ t \in T}\sum\limits_{ n \in N}z_{i,t,n}^{f,s} = 1 \hspace{20 pt} \forall f \in  F_s^+, \forall s  \in  S$

In [32]:
model.del_component( 'MappingVNFRequest')
model.del_component( 'MappingVNFRequest_Request')

def MappingVNFRequest_rule(model,s, f):
    return sum(model.z[s,f,t,m,c] for t,m in model.TM for c in model.Nc) == 1

model.MappingVNFRequest = Constraint(model.SF, rule=MappingVNFRequest_rule)

In [33]:
# model.MappingVNFRequest.display()

##### C8: VNF Assignment

A VNF service reuested by a SFC can be assigned to an instance of a VNF at a server if this is already deployed and is of the requested type

$z_{i,t,n}^{f,s} \leq x_{i,t,n}  \cdot  T_{s}^{f,t}
    \hspace{20 pt} \forall  f  \in  F_s^+, \forall s  \in  S,  \forall i  \in  I_t, \forall t  \in  T,  \forall n  \in  N$

In [34]:
model.del_component( 'VNFAssignment')
model.del_component( 'VNFAssignment_index')
model.del_component( 'VNFAssignment_index_index_0')
model.del_component( 'VNFAssignment_index_index_0_index_0')
model.del_component( 'VNFAssignment_index_index_0_index_0_index_0')

#original version    
def VNFAssignment_rule(model,s,f,t,m,c):
    if t==Ran_type:
        if (s,f,t,m,c) in List_R_s:                 # assign s to its source ran
            return model.z[s,f,t,m,c]==1
        else:
            return model.z[s,f,t,m,c]==0
    else:
        return model.z[s,f,t,m,c] <= model.x[t,m,c]*model.T_sft[s,f,t]

model.VNFAssignment = Constraint(model.SF_extended, model.TM_extended, model.N, rule=VNFAssignment_rule)

In [35]:
# model.VNFAssignment.display()

##### C9: Minimum number of VNF of each Type Assigned to a session
Each SFCR must be assigned to at least the minimun number of requested VNFs of eac type

$ \sum\limits_{ f \in F_s}  \sum\limits_{ i \in I_t}\sum\limits_{ n \in N_c}z_{i,t,n}^{f,s} \geq |F_s^t| 
    \hspace{20 pt}  \forall s  \in  S, \forall t  \in  T; t\neq 0 $

In [36]:
model.del_component( 'MinNUmVNFTypeAssignment')
model.del_component( 'MinNUmVNFTypeAssignment_index')

def MinNUmVNFTypeAssignment_rule(model,s, t):
    return sum(model.z[s,f,t,m,c] for f in model.F[s] for m in model.M_t[t] for c in model.Nc) >= model.Num_VNFTypes_s[s,t]

model.MinNUmVNFTypeAssignment = Constraint(model.S, model.T,  rule=MinNUmVNFTypeAssignment_rule)

#### C10: VNF_Branch 

Express that a VNF f in m,t,c can serve a branch b of a SFCR s if it is assigned to s. Or ...

$z_{m,t,c}^{s,f} \leq \sum\limits_{b \in B_s} a_{m,t,c}^{s,b,f}  \hspace{20 pt} \forall s \in S, \forall f \in F_s, \forall t \in T,  \forall m \in M_t, \forall c \in Nc$

In [39]:
model.del_component( 'VNF_Branch')
model.del_component( 'VNF_Branch_index')
model.del_component( 'VNF_Branch_index_index_0')
model.del_component( 'VNF_Branch_index_index_0_index_0')
model.del_component( 'VNF_Branch_index_index_0_index_0_index_0')

def VNF_Branch_rule(model, s,f,t,m,c):
    return model.z[s,f,t,m,c] <= sum(model.a[s,b,f,t,m,c] for b in model.B_s[s])

model.VNF_Branch = Constraint(model.SF, model.TM, model.Nc, rule= VNF_Branch_rule)
# model.VNF_Branch = Constraint(model.SF_extended, model.TM_extended, model.N, rule= VNF_Branch_rule)

#### C11: a and Z variables

Express that a VNF can serve a branch only if it is placed at a location

$ a_{m,t,c}^{s,b,f} \leq z_{m,t,c}^{s,f} \hspace{20 pt} \forall s \in S, \forall f \in F_s, \forall t \in T,  \forall m \in M_t, \forall c \in Nc$

In [41]:
model.del_component( 'A_Z')
model.del_component( 'A_Z_index')
model.del_component( 'A_Z_index_index_0')
model.del_component( 'A_Z_index_index_0_index_0')
model.del_component( 'A_Z_index_index_0_index_0_index_0')
model.del_component( 'A_Z_index_index_0_index_0_index_0_index_0')

def A_Z_rule(model, s,b,f,t,m,c):
    return model.z[s,f,t,m,c] >= model.a[s,b,f,t,m,c]

# model.A_Z = Constraint(model.SBF, model.TM, model.Nc, rule= A_Z_rule) # no funciona
model.A_Z = Constraint(model.SBF_extended, model.TM_extended, model.N, rule= A_Z_rule)

#### C12: VNFBranchPresence
Express if a VNF f is present in a branch b of a SFCR s 

$\sum\limits_{m \in M_t} \sum\limits_{c \in N_c} a_{m,t,c}^{s,b,f} \leq Q_{sbf} \hspace{20 pt} \forall b \in B_s,\forall s \in S, \forall f \in F_s, \forall t \in T$

In [43]:
model.del_component( 'VNFBranchPresence')
model.del_component( 'VNFBranchPresence_index')
model.del_component( 'VNFBranchPresence_index_index_0')

#original version
def VNFBranchPresence_rule(model, s,b,f,t):
    return sum(model.a[s,b,f,t,m,c] for m in model.M_t[t] for c in model.N) <= model.P_sbf[s,b,f]*model.T_sft[s,f,t]

model.VNFBranchPresence = Constraint(model.SBF_extended, model.T_extended, rule=VNFBranchPresence_rule)


#### C13 : Link_Order

$  \sum \limits_{ p \in P}  y_{p}^{f,g,b,s}  \geq O_{s}^{f,g,b} \hspace{20 pt} \forall f, g \in  F_s^+, \forall b  \in  B_s, \forall s  \in  S$

In [45]:
model.del_component( 'LinkOrder1')
model.del_component( 'LinkOrder1_index')
model.del_component( 'LinkOrder1_index_index_0')
model.del_component( 'LinkOrder1_index_index_0_index_0')

def LinkOrder1_rule(model, s,b,f,g):
    return model.O_sbfg_extended[s,b,f,g] <= sum(model.y[s,b,f,g,n,m,h] for n,m,h in model.Paths)

model.LinkOrder1 = Constraint(model.SBFG_extended, rule= LinkOrder1_rule)

#### C15: VNF_link

$ \sum \limits_{ p \in P_{n,m}}  y_{p}^{f,g,b,s} \leq \sum\limits_{ i \in I_t}\sum\limits_{ t \in T}a_{i,t,n}^{f,b,s} \cdot \sum\limits_{ i \in I_t}\sum\limits_{ t \in T} a_{t,i,m}^{s,b,g} \hspace{20 pt} \forall f, g \in  F_s^+,  \forall b  \in  B_s, \forall s  \in  S, \forall n,m  \in  N $

In [49]:
# Linear form of the constraint

model.del_component( 'VNF_link1')
model.del_component( 'VNF_link1_index')
model.del_component( 'VNF_link1_index_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0_index_0_index_0')

def VNF_link1_rule(model, s,b,f,g,n,m):
    return sum(model.a[s,b,f,t,i,n] for t,i in  model.TM_extended)>=model.delta[s,b,f,g,n,m]

model.VNF_link1 = Constraint(model.SBFG_extended, model.Paths_nm, rule= VNF_link1_rule)

model.del_component( 'VNF_link2')
model.del_component( 'VNF_link2_index')
model.del_component( 'VNF_link2_index_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0_index_0_index_0')

def VNF_link2_rule(model, s,b,f,g,n,m):
    return sum(model.a[s,b,g,t,i,m] for t,i in model.TM_extended)>=model.delta[s,b,f,g,n,m]
    
model.VNF_link2 = Constraint(model.SBFG_extended,  model.Paths_nm, rule= VNF_link2_rule)

model.del_component( 'VNF_link3')
model.del_component( 'VNF_link3_index')
model.del_component( 'VNF_link3_index_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0_index_0_index_0')

def VNF_link3_rule(model, s,b,f,g,n,m):
    if f!=g:
        return model.delta[s,b,f,g,n,m] >= sum(model.a[s,b,f,t,i,n] + model.a[s,b,g,t,i,m] for t,i
                                               in model.TM_extended)-1
    else:
        return model.delta[s,b,f,g,n,m] <= 0
    
model.VNF_link3 = Constraint(model.SBFG_extended, model.Paths_nm, rule= VNF_link3_rule)

model.del_component( 'VNF_link')
model.del_component( 'VNF_link_index')
model.del_component( 'VNF_link_index_index_0')
model.del_component( 'VNF_link_index_index_0_index_0')
model.del_component( 'VNF_link_index_index_0_index_0_index_0')
model.del_component( 'VNF_link_index_index_0_index_0_index_0_index_0')

def VNF_link_rule(model, s,b,f,g,n,m):
    return model.delta[s,b,f,g,n,m] >= sum(model.y[s,b,f,g,n,m,h] for h in model.Paths_nm_h[n,m])

model.VNF_link = Constraint(model.SBFG_extended, model.Paths_nm, rule= VNF_link_rule)

#### C16: Anti-Affinity
The VNFs of a same type requested by a same SFCR must be mapped in different servers. In other words, two VNFs of a same time serving a same SFCR must be placed on different servers

$ \sum \limits_{f \in F_s}  \sum \limits_{i \in I_t}  z_{i,t,n}^{f,s} \leq 1 \hspace{20 pt} \forall s  \in  S, \forall t  \in  T; t \neq 0,\forall n  \in  N_c$

In [50]:
model.del_component( 'Anti_Affinity')
model.del_component( 'Anti_Affinity_index')
model.del_component( 'Anti_Affinity_index_index_0')

def Anti_Affinity_rule(model, s,c,t):
    return sum(model.z[s,f,t,i,c] for f in model.F[s] for i in model.M_t[t])<= 1

model.Anti_Affinity = Constraint(model.S, model.Nc, model.T, rule= Anti_Affinity_rule)

#### C17: I-UPF and A-UPF

The UPFs of type I-UPF  and PSA must not be co-located in the same servers

$  \sum \limits_{f \in F_s} \sum \limits_{i \in I_t}  z_{1,i,n}^{s,f} + \sum \limits_{f \in F_s}   \sum \limits_{i \in I_t}  z_{3,i,n}^{s,f} \leq 1  \hspace{20 pt}  \forall s  \in  S, \forall n  \in  N_c $

In [52]:
model.del_component( 'IUPF_PSA')
model.del_component( 'IUPF_PSA_index')

def IUPF_PSA_rule(model, s,c):
    return sum(model.z[s,f,1,m,c] for f in model.F[s] for m in model.M_t[1])+sum(model.z[s,f,2,m,c] for f in model.F[s] for m in model.M_t[2]) <= 1

model.IUPF_PSA = Constraint(model.S, model.Nc, rule= IUPF_PSA_rule)

#### C18: Latency Requirement

The round trip data plane delay of a PDU session cannot overpass its service latency requirement (L_s)

$ 2 \cdot  \left(  \sum \limits_{f \in F_s^+}\sum \limits_{i \in I_t} \sum \limits_{t \in T}\sum \limits_{n \in N}  d_t  \cdot  a_{i,t,n}^{f,b,s} +\sum \limits_{f,g  \in  F_s^+} \sum\limits_{ p \in P}  d_{p}  \cdot  y_{p}^{f,g,b,s}\right) + 
d_{DN}  \leq L_s  \hspace{20 pt}  \forall b  \in  B_s, \forall s  \in  S $


In [54]:
model.del_component( 'Latency')
model.del_component( 'Latency_index')

def Latency_rule(model,s,b):
    latency= 2*sum(model.a[s,b,f,t,i,c]*model.D_proc[t] for t,i in model.TM for c in model.Nc 
                for f in model.F[s])+ 2*sum(model.D_prop[n,m,h]*model.y[s,b,f,g,n,m,h] for n,m,h in model.Paths 
                                          for f in model.F[s] for g in model.F[s])+2*sum(model.D_prop[n,m,h]*model.y[s,b,model.Num_VNF_s[s],1,n,m,h]
                                                                                         for n,m,h in model.Paths)+T_dn+model.D_proc[Ran_type] 
    return latency <= model.Lreq_s[s]

model.Latency = Constraint(model.SB, rule=Latency_rule)

## SOLVING THE MODEL AND GETTING THE RESULTS

In [58]:
# solve the model
start_t_process = time.process_time()  #Process time for profiling: sum of the kernel and user-space CPU time.

# solver = SolverFactory('glpk')
# solver = SolverFactory('cplex')
solver = SolverFactory('gurobi')
# solver.options['mipgap'] = 0.01       # uncomment to set an optimility gap
results= solver.solve(model)

T_exec= time.process_time() - start_t_process       # SOLUTION TIME

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('   OK\n')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('   INFEASIBLE\n')
else:
    # Something else is wrong
    print ("   Solver Status: ",  result.solver.status,'\n')


   OK



In [60]:
cost = model.Objective.expr()

### Getting the results

In [61]:
# Number of VNFs
Total_upf = 0
Num_upf_type = {}

for t in model.T:
    t_upf = 0
    for m in model.M_t[t]:
        for c in model.Nc:
            if model.x[t,m,c].value > 0.9:
                Total_upf += 1
                t_upf += 1
    Num_upf_type[t] = t_upf

In [62]:
# Number of open servers
Num_open_c = 0
Open_servers_list = []

for c in model.Nc:
    if model.w[c].value > 0.9:
        Open_servers_list.append(c)
        Num_open_c += 1

In [63]:
Latency_List = []

for s,b in model.SB:
    latency= 2*sum(model.a[s,b,f,t,i,c].value*model.D_proc[t] for t,i in model.TM for c in model.Nc 
                for f in model.F[s])+ 2*sum(model.D_prop[n,m,h]*model.y[s,b,f,g,n,m,h].value
                                            for n,m,h in model.Paths for f in model.F[s] 
                                            for g in model.F[s])+2*sum(
        model.D_prop[n,m,h]*model.y[s,b,model.Num_VNF_s[s],1,n,m,h].value for n,m,h in model.Paths)+T_dn+model.D_proc[Ran_type] 
    Latency_List.append(latency)


In [64]:
print('   Execution time', T_exec)
print('   Cost', cost)
print('   Number of UPF installed: ', Total_upf )
print('   Number of UPFs per type', Num_upf_type )
print('   Open servers list', Open_servers_list )
print('   Number of Open Servers', Num_open_c )
print('   Maximum latency', max(Latency_List) )

   Execution time 381.09375
   Cost 0.1282048796282985
   Number of UPF installed:  4
   Number of UPFs per type {1: 2, 2: 1, 3: 1}
   Open servers list [8, 9]
   Number of Open Servers 2
   Maximum latency 910.12
